In [15]:
from lyricsgenius import Genius
from bs4 import BeautifulSoup
import requests
import pandas as pd
from itertools import islice
import os
import re
from tqdm.notebook import tqdm
import time

In [4]:
# Authenticate with Genius API
genius = Genius('FeuRaslWUfDgOWiEO3QS6l2vmWY5i0oHH4fiMtz7-LLuYy8NR5JFkTc92ZwQgmR-', verbose=False, timeout=20)

In [29]:
def get_top50_from_url(year):
    #example = url_start + '2018' + url_end
    url_start = 'https://www.billboard.com/charts/year-end/'
    url_end = '/hot-rap-songs/'

    url = url_start + str(year) + url_end
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('div', {'class': 'o-chart-results-list-row-container'})

    data = []

    for i in range(len(results)):
        result = results[i]
        title = result.find('h3').text.strip()
        spans = result.find_all('span')
        artist = spans[1].text.strip()
        data.append({'position': i+1, 'title': title, 'artist': artist, 'year': year})

    df = pd.DataFrame(data)
    return df

In [46]:
def get_lyrics_and_primary_artist(df):
    lyrics = []
    primary_artists = []
    for i, row in tqdm(df.iterrows(), desc="songs", position=0, leave=True):
        artist = row["artist"]
        title = row["title"]

        genius_fetch = genius.search_song(title, artist)
        # check whether its the translated version....
        if genius_fetch is not None and "Genius" in genius_fetch.primary_artist.name:
            # try alternative
            artist = artist.lower()
            if "featuring" in artist:
                a = artist.split("featuring")[0]
                genius_fetch = genius.search_song(title, a)
            elif "ft." in artist:
                a = artist.split("ft.")[0]
                genius_fetch = genius.search_song(title, a)

        if genius_fetch is None:
            lyrics.append(None)
            primary_artists.append(None)
        else:
            lyrics.append(genius_fetch.lyrics)
            primary_artists.append(genius_fetch.primary_artist.name)

    return lyrics, primary_artists

In [28]:
# cleans lyrics
def clean_lyrics(lyrics):
    # remove everything until the first occurence of Lyrics (including this word)
    lyrics = re.sub(r'^.*?Lyrics', '', lyrics)
    #
    lyrics = lyrics.replace("\u2005"," ")
    # remove all tags ( tag: [ XX ]
    lyrics = re.sub(r'\[.+?\]','',lyrics)
    # the last word contains 000Embeded, where 000 is any number. remove this
    lyrics = re.sub(r"\d*Embed$",'', lyrics)
    # no need for \n
    lyrics = lyrics.replace("\n"," ")
    # remove multiple spaces
    lyrics = re.sub('\s+',' ', lyrics).strip()
    return lyrics

In [49]:
dfs = []
for y in tqdm(range(2018,2023), desc="Year"):
    # get top 50 rap songs for given year
    df = get_top50_from_url(y)
    # get the primary artists and cleaned lyrics for each song
    lyrics, primary_artists = get_lyrics_and_primary_artist(df)
    # clean the lyrics
    cleaned_lyrics = []
    for l in lyrics:
        if l is None:
            cleaned_lyrics.append(None)
        else:
            cleaned_lyrics.append(clean_lyrics(l))
    # add them to df
    df["Lyrics"] = cleaned_lyrics
    df["Primary Artists"] = primary_artists

    dfs.append(df)

Year:   0%|          | 0/5 [00:00<?, ?it/s]

songs: 0it [00:00, ?it/s]

songs: 0it [00:00, ?it/s]

songs: 0it [00:00, ?it/s]

songs: 0it [00:00, ?it/s]

songs: 0it [00:00, ?it/s]

In [50]:
df_all = pd.concat(dfs)
df_all

,position,title,artist,year,Lyrics,Primary Artists
0,1,God's Plan,Drake,2018,And they wishin' and wishin' And wishin' and w...,Drake
1,2,I Like It,"Cardi B, Bad Bunny & J Balvin",2018,"Yeah, baby, I like it like that You gotta beli...","Cardi B, Bad Bunny & J Balvin"
2,3,In My Feelings,Drake,2018,"Trap, TrapMoneyBenny This shit got me in my fe...",Drake
3,4,Psycho,Post Malone Featuring Ty Dolla $ign,2018,"Damn, my AP goin' psycho, lil' mama bad like M...",Post Malone
4,5,Nice For What,Drake,2018,I wanna know who mothafuckin' representin' in ...,Drake
...,...,...,...,...,...,...
45,46,Already Dead,Juice WRLD,2022,"You can see the pain in my laugh (Yeah, yeah) ...",Juice WRLD
46,47,Family Ties,Baby Keem & Kendrick Lamar,2022,"Jump in that— (Hah, Hah), summon that bitch Ju...",Baby Keem & Kendrick Lamar
47,48,Alone,Rod Wave,2022,B Squared beats Oh (Oh-woah) Oh-woah Tell me w...,Rod Wave
48,49,Plan B,Megan Thee Stallion,2022,"Who the fuck you think you talkin' to, nigga? ...",Megan Thee Stallion


In [51]:
df_all.to_csv('top50_2018_2022.csv', index=False)